In [9]:
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import default_rng

import simpy

## Improving our initial vaccine clinic model

Now that we have a rough first model working, let's think about some possible improvements. There are many as we've taken numerous shortcuts to get this model working.

* Specifying the global sim inputs through configuration files
* Getting rid of hard coded processing time distributions
* Having ability to choose between pure random walk-in arrivals and scheduled arrivals
* Multiple replications and/or steady state analysis
* Detailed logging
* Statistical summaries based on timestamp data [roughly done]
* CLI for running
* Animation


## Redesign for easier use

### Hard coded input parameters

The current version of the model has many hard coded input parameters, including:

* patient arrival rate,
* percent of patients requiring 2nd dose,
* capacity of the various resources (e.g. vaccinators),
* processing time distributions for the various stages of the vaccination process,
* length of time patient needs to remain in observation after being vaccinated (i.e. the 15 minutes),
* length of time to run the simulation model.

This makes it cumbersome to try out different scenarios relating to patient volume and resource capacity.

### Better post-processing

We should make it easy to specify that some post-processing should just happen automatically. In general, we want better control over post-processing and think through how we want to create and store output files.

### Moving code from Jupyter notebook to a Python script

As our code grows, working inside a Jupyter notebook becomes less practical. It's fine for testing out little code snippets, but the production code should be in a `.py` file and we should use an IDE for further development. I've already used the Jupyter notebook export feature to export Model 3 to `vaccine_clinic_model4.py` that we'll use as the basis for further development.

### Adding a command line interface

By moving the code to a `.py` file, it will also be easier to add and use a simple command line interface. This will be useful for automating the process of running numerous scenarios.


## Creating a config file for input parameters (and creating a CLI)

These two enhancements are actually related and we'll address them together. Instead of hard coding input parameter values into our code, we could store them in a simple configuration file. Before getting into configuration files, we need to step back and review passing command line arguments to Python scripts.

### Basic handling of command line arguments (learning about `argv`)

Back in the pcda class, we used some materials from the [Software Carpentry Python Programming lesson](). There were several parts that we left as optional, including the last part on [Command Line Programs](https://swcarpentry.github.io/python-novice-inflammation/12-cmdline/index.html). I highly encourage you to review that first. 

**Bottom line:** `sys.argv` is a list of command line arguments passed to a Python program when running the program.

Let's create a small program that takes a few command line arguments and just repeats them back when the program is run. Here's what the program looks like. I've saved it as `echo_args.py`. We'll interpret the command line arguments as follows:

1. mean interarrival times of patients
2. number of greeters
3. number of registration staff
4. number of vaccinators
5. number of schedulers

We call these *positional arguments* in that our program will infer the meaning of each passed in argument from its position on the command line.

In [ ]:
# echo_args.py
import sys

def main():
    print(f"Command line args: {sys.argv}\n")

    for i, arg in enumerate(sys.argv):
            print(f"sys.argv[{i}]: {arg}")

if __name__ == '__main__':
   main()

In [1]:
__name__

'__main__'

If you've forgotten what the purpose of the following is:

```
if __name__ == '__main__':
```

then, make sure you review the [Command Line Programs](https://swcarpentry.github.io/python-novice-inflammation/12-cmdline/index.html) mentioned above. In a nutshell, when a Python program is run (as opposed to being imported), the special Python variable `__name__` is equal to `'__main__'`. When a Python program is imported, `__name__` is equal to the name of the module. So, the code snippet above is often included so that a Python program can be both run and imported.

Also notice the use of `enumerate` with `sys.argv` which allows us to get both the index and argument value from the `sys.argv` list - no need to make our own index loop.

In [11]:
!python ../src/vaccine_clinic/echo_args.py 3.0 2 4 15 4

Command line args: ['../src/vaccine_clinic/echo_args.py', '3.0', '2', '4', '15', '4']

sys.argv[0]: ../src/vaccine_clinic/echo_args.py
sys.argv[1]: 3.0
sys.argv[2]: 2
sys.argv[3]: 4
sys.argv[4]: 15
sys.argv[5]: 4


So, `sys.argv` is a list of the command line arguments passed in to `echo_args.py`. Note that `sys.argv[0]` is just the name of the program itself, including the relative path from this working directory to the program file. At this point, our program doesn't do any checking of the presence or validity of the input arguments. It just prints back out whatever values we input on the command line. If all of these arguments were required, we could add a check like this:

In [2]:
# echo_args.py
import sys

def main():
    print(f"Command line args: {sys.argv}\n")

    if len(sys.argv) != 6:
        print(f"Five args required, only {len(sys.argv) - 1} specified.")
        exit(1)

    for i, arg in enumerate(sys.argv):
            print(f"sys.argv[{i}]: {arg}")

if __name__ == '__main__':
   main()

Command line args: ['C:\\Users\\isken\\Anaconda3\\envs\\aap\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\isken\\AppData\\Roaming\\jupyter\\runtime\\kernel-012eae39-d6c6-4fff-9e76-b5148246eb6a.json']

Five args required, only 2 specified.
sys.argv[0]: C:\Users\isken\Anaconda3\envs\aap\lib\site-packages\ipykernel_launcher.py
sys.argv[1]: -f
sys.argv[2]: C:\Users\isken\AppData\Roaming\jupyter\runtime\kernel-012eae39-d6c6-4fff-9e76-b5148246eb6a.json


In [3]:
!python ../src/vaccine_clinic/echo_args.py 3.0 2 4

Command line args: ['../src/vaccine_clinic/echo_args.py', '3.0', '2', '4']

Five positional args required, only 3 specified.


The example above is highly simplified and the world of command line arguments and command line processing, known as *argument parsing*, is much richer than this. Not only do we have *arguments* such as in this example, we also might have *options* (also called *flags*). In the pcda class we saw this when using things like the `ls` command:

```
ls -l -a
```

We might want to define options for our Python program. These options might appear in any order, if at all. Furthermore,
by convention there are short form flags that start with a single `-` such as `-a` in the example above, and long form options that start with `--` such as `--version`. Often we can use either a short or long form option such as `-h` or `--help`. 

Before checking out argument parsing tools, let's do it ourselves for the following simple case. Let's assume we just want the following command line options. They can be in any order but each must be followed by a numeric value for that option. So,
`argv[i]` will be one of the following for odd values of `i` and `argv[i + 1]` will be the corresponding value.

* `--iat` : mean patient interarrival time
* `--greet` : number of greeters
* `--reg` : number of registration clerks
* `--vacc` : number of vaccinators
* `--sched` : number of schedulers

The following is just a code snippet to illustrate how one might get the input values using standard Python. Once we have the input values, we could pass them on to other parts of our simulation model. Notice that this code doesn't do any input validity checking other than checking for invalid option names. The user input values are stored in a dictionary and this code just prints out that dictionary.

In [4]:
# get_option_values.py
import sys

def main():

    input_params = {'mean_interarrival_time': 0.0,
                    'num_greeters': 0,
                    'num_reg_staff': 0,
                    'num_vaccinators': 0,
                    'num_schedulers': 0}


    for i, arg in enumerate(sys.argv):
        if arg.startswith('--') and i % 2 > 0:
            if sys.argv[i] == '--iat':
                input_params['mean_interarrival_time'] = sys.argv[i + 1]
            elif sys.argv[i] == '--greet':
                input_params['num_greeters'] = sys.argv[i + 1]
            elif sys.argv[i] == '--reg':
                input_params['num_reg_staff'] = sys.argv[i + 1]
            elif sys.argv[i] == '--vacc':
                input_params['num_vaccinators'] = sys.argv[i + 1]
            elif sys.argv[i] == '--sched':
                input_params['num_schedulers'] = sys.argv[i + 1]
            else:
                print(f"Unrecognized argument: {sys.argv[i]}")

    print(input_params)

if __name__ == '__main__':
   main()

{'mean_interarrival_time': 0.0, 'num_greeters': 0, 'num_reg_staff': 0, 'num_vaccinators': 0, 'num_schedulers': 0}


In [5]:
!python ../src/vaccine_clinic/get_option_values.py --iat 3.0 --greet 2 --reg 4 --vacc 15 --sched 4

{'mean_interarrival_time': '3.0', 'num_greeters': '2', 'num_reg_staff': '4', 'num_vaccinators': '15', 'num_schedulers': '4'}


Here we change the order and include one bad option.

In [6]:
!python ../src/vaccine_clinic/get_option_values.py --vacc 15 --iat 3.0 --greet 2 --reg 4 --schedulers 4

Unrecognized argument: --schedulers
{'mean_interarrival_time': '3.0', 'num_greeters': '2', 'num_reg_staff': '4', 'num_vaccinators': '15', 'num_schedulers': 0}


#### Learning more about command line arguments
If you are interested in a deeper dive into command line arguments, check out this tutorial done by the folks at Real Python:

* [Python Command Line Arguments](https://realpython.com/python-command-line-arguments/)

### Tools for argument parsing and building CLI's.

Clearly, it is going to be (potentially) difficult to deal with all the complexity of command line *argument parsing* manually by checking all the values in the `sys.argv` list. Thankfully, there are numerous tools for doing command line argument parsing and helping us create command line interfaces for our Python programs. Some of these tools include:

* [argparse](https://docs.python.org/3/library/argparse.html) - part of the Python standard library
* [click](https://click.palletsprojects.com/en/7.x/) - a popular library for CLIs that uses [function decorators]()
* [fire](https://github.com/google/python-fire) - a newer CLI tool

For our simulation model, we'll use `argparse` to build our CLI since it's built in to Python and is a good thing to learn if you are new to creating CLIs. It's plenty powerful enough for our simple application. Some learning resources for `argparse` include:

* [A "gentle" argparse tutorial](https://docs.python.org/3/howto/argparse.html#id1)
* https://docs.python.org/3/library/argparse.html - the official docs
* https://realpython.com/command-line-interfaces-python-argparse/ - tutorial from Real Python (different that the one on Command Line Arguments mentioned above - this one focuses on argparse)

### Creating a CLI with argparse
As discussed in the tutorials above, there are four main steps to creating a CLI with argparse.

1. Import the argparse library
2. Create a parser object
3. Add arguments of the desired types to the parser
4. Use it by calling the `arg_parse` method of the parser object

After calling `arg_parse` you'll get what is known as a [Namespace object](https://docs.python.org/dev/library/argparse.html#argparse.Namespace).

I rewrote the `echo_args.py` example using argparse. You can find it in `echo_args_argparse.py`. It just prints out `args` (the Namespace object) and `vars(args)` which gives a dictionary version of the arguments and their values.

In [13]:
!python ../src/vaccine_clinic/test_argparse.py --iat 3.0 --greet 2 --reg 4 --vacc 15 --sched 4

args:  Namespace(greet=2, iat=3.0, reg=4, sched=4, vacc=15)
vars(args): {'iat': 3.0, 'greet': 2, 'reg': 4, 'vacc': 15, 'sched': 4}


Now that we've got a basic grasp of creating a simple CLI, we will do all further development on this model in the `vaccine_clinic_model4.py`. 